In [ ]:
import networkx as nx
import osmnx as ox
import requests
import math
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
import numpy as np
import sys
import pickle
from IPython.display import clear_output
from operator import itemgetter
import os
%matplotlib inline
ox.config(use_cache=True, log_console=True)
ox.__version__

# Global variables

In [ ]:
num_obs = 100

# norm features of the model which are individual for each mode of transportation and used for the calculation of the uc
norm_mod_features = ["norm_travel_time", "norm_discomfort", "norm_price"]

# norm features of the model which are independet of the mode of transportation and used for the calculation of the uc
norm_features = ["norm_co2"]

# not normalized value of the features dependent of modality: used for initialization of normalized values
nnm_features = ["travel_time", "discomfort", "price"]

# not normalized value of the features independent of modality: used for initialization of none normalized values
nn_features = ["co2"]

# all mode of transportations
mode = ["car", "bike", "walk", "public"]

# all beta_features applying to normalized features
beta_features = ["norm_co2", "norm_num_trans",
                 "norm_travel_time_car", "norm_travel_time_bike", "norm_travel_time_walk", "norm_travel_time_public",
                 "norm_discomfort_car", "norm_discomfort_bike", "norm_discomfort_walk", "norm_discomfort_public",
                 "norm_price_car", "norm_price_bike", "norm_price_walk", "norm_price_public"]

# file path for preprocessed dataframes
try:
    from src.a_star import prep_dataframes  #note: MMWPF path must be added to enviornmental variables PYTHONPATH (see setup.py)
    prep_path = prep_dataframes.path()
except:
    prep_path = os.path.join("./", "prep_dataframes")
    if not os.path.exists(prep_path):
        print('%s path does not exist!' % prep_path)

# Preprocessing edge and node network

In [ ]:
try:
    from src.maps import graphML  #note: MMWPF path must be added to enviornmental variables PYTHONPATH (see setup.py)
    composed_graph_path = graphML.path()
except:
    composed_graph_path = os.path.join("./", "comp_graph")
    if not os.path.exists(composed_graph_path):
        print('%s path does not exist!' % composed_graph_path)

# hardcoded path - needs to be adjusted
g_drive_path = os.path.join(composed_graph_path, "G_drive.graphml")
g_bike_path = os.path.join(composed_graph_path, "G_bike.graphml")
g_walk_path = os.path.join(composed_graph_path, "G_walk.graphml")
g_public_path = os.path.join(composed_graph_path, "G_public.graphml")
g_all_path = os.path.join(composed_graph_path, "G_all.graphml")

G_drive = ox.load_graphml(g_drive_path)
G_bike = ox.load_graphml(g_bike_path)
G_walk = ox.load_graphml(g_walk_path)
G_public = ox.load_graphml(g_public_path)
G_all = ox.load_graphml(g_all_path)
G_all = ox.project_graph(G_all)

lst_g = [G_drive, G_bike, G_walk, G_public]

In [ ]:
# G_all = nx.compose(G_drive, G_bike)
# G_all = nx.compose(G_all, G_walk)
# G_all = nx.compose(G_all, G_public)

In [ ]:
# walking speed is way too high, need so scale it so it is between 3 - 10
def rescale_walk_speed(spd):
    if spd <= 10.0:
        return spd
    elif spd <= 20.0:
        return spd / 2.0
    elif spd <= 30.0:
        return spd / 3.0
    elif spd <= 40.0:
        return spd / 4.0
    elif spd <= 50.0:
        return spd / 5.0
    elif spd <= 60.0:
        return spd / 6.0
    elif spd <= 70.0:
        return spd / 7.0
    elif spd <= 80.0:
        return spd / 8.0

In [ ]:
lst_nodes = []
lst_edges = []

# the order of the network is: 1. drive, 2. bike, 3. walk, 4. public
for mode_idx, G in enumerate(lst_g):
    # Reproject the data from a WGS84 format into a metric system 
    G_tmp_proj = ox.project_graph(G)
    G_tmp_proj = ox.add_edge_speeds(G_tmp_proj)
    G_tmp_proj = ox.add_edge_travel_times(G_tmp_proj)
    g_node_tmp, g_edge_tmp = ox.graph_to_gdfs(G_tmp_proj)
    # mode_idx 2 is walk mode
    # only the speed of for walk needs to be adjusted
    if mode_idx == 2:
        g_edge_tmp["speed_kph"] = g_edge_tmp["speed_kph"].apply(rescale_walk_speed)
        # recalculate travel time based on the new speed_kph
        g_edge_tmp["travel_time"] = g_edge_tmp["length"] / (g_edge_tmp["speed_kph"] * 1000 / 3600)
    # mode_idx 3 is public mode
    elif mode_idx == 3:
        # drop column in order to concatenate them later on
        g_node_tmp.drop(["stopid", "color"], axis=1, inplace=True)
    # rescale speed to meter per second
    g_edge_tmp["speed_ms"] = g_edge_tmp["speed_kph"] * 1000 / 3600
    
    ###### Comment the below code out, if your osmnx version is older than 1.0.1 ######
    # unstack the multiindex
#     g_node_tmp.drop("osmid", axis=1, inplace=True)
#     g_node_tmp =  g_node_tmp.reset_index()
#     g_node_tmp.index = g_node_tmp.osmid
#     g_node_tmp.index.rename('', inplace=True)
    ###### Comment the above code out, if your osmnx version is older than 1.0.1 ######

    g_edge_tmp = g_edge_tmp.reset_index()

    lst_nodes.append(g_node_tmp)
    lst_edges.append(g_edge_tmp)

In [ ]:
# utiliy function for edge transformation
def indices(mylist, value):
    return [i for i,x in enumerate(mylist) if x[1]==value]

# if there are 2 same edges with different number than they are stored in a list
def tran_mult_edges(lst):
    # Some nodes don't have any other adjacent edges and returns a NaN value
    # for those nodes just return NaN value
    if lst != lst:
        return lst
    trg_lst = [l[1] for l in lst]
    if len(set(trg_lst)) == len(trg_lst):
        return lst
    else:
        for trg in set(trg_lst):
            idc = indices(lst, trg)
            # if no multiple edges for the target value then return
            if len(idc) > 1:
                first_idx = idc[0]
                t = [lst[idx] for idx in idc]
                for idx in sorted(idc, reverse = True):  
                    del lst[idx]
                lst.insert(first_idx, t)
            else:
                continue
        return lst
    
def calc_discomfort(length, mode):
    d = length
    if mode == "car":
        d = length
    elif mode == "bike":
        d = 2.0 * length
    elif mode == "walk":
        d = 3.0 * length
    elif mode == "public":
        d = 1.2 * length  
    return d

def calc_co2(length, mode):
    # ref --> 127g / km
    if mode == "car":
        co2 = length * 127 / 1000
    # 21g / km
    elif mode == "bike":
        co2 = length * 21 / 1000
    # 5g / km
    elif mode == "walk":
        co2 = length * 5 / 1000
    # bus = 75, ubahn = 30.5, train = 28 --> avg = 44.5 / km
    elif mode == "public":
        co2 = length * 39.125 / 1000 
    return co2

# public is calculated in a lambda function since it uses length instead of travel_time
def calc_price(travel_time, mode):
    # ref --> sixt share start at 0.09 per minute, depends on offer/demand --> assume in average a cost of 0.12 per minute
    if mode == "car":
        pr_cost = 0.002
    elif mode == "bike":
        pr_cost = 0.0013 
    elif mode == "walk":
        pr_cost = 0
    # day ticket in cost per second: 7.9 per day
    elif mode == "public":
        pr_cost = 0.0001
    # opportunity cost of 8€ / hour
    op_cost = 8 / 3600
    price = (op_cost + pr_cost) * travel_time
    return price

In [ ]:
################ edge transformations ################
 
def select_add_transform_attributes(frames, scaler):
    filtered_columns = ["osmid", "highway", "length", "speed_ms", "travel_time", "u", "v", "key"]
    # order of the list: [df_car, df_bike, df_walk]
    # only select the specified columns
#     lst_f = [f.loc[:,columns] for f in frames]
    lst_f = [f.reindex(columns=filtered_columns, index=None) for f in frames]
    
    # store the index of each df in order to split them up again later on
    lst_idx = []
    next_idx = 0
             
    # add column for each cost category by transforming them into a dictionary
    # where each dataframe has the mode of transportation as key
    # increment count by one after each f to select the corresponding mode of transportation
    # mode = ["car", "bike", "walk", public] --> e.g. mode[0] == car

    for count, f in enumerate(lst_f):
        f["co2"] = f["length"].apply(calc_co2, mode=mode[count]) 
        f["discomfort"] = f["length"].apply(calc_discomfort, mode=mode[count])
        f["price"] = f["travel_time"].apply(calc_price, mode=mode[count])
        # only used for initilization
        for att in beta_features:
            f[att] = 0.0
        
        # save the last index of each df
        # after each iteration add the last index value to it in order to extract it later on 
        next_idx += f.index.stop
        lst_idx.append(next_idx)
    
    # combine all frames for rescaling
    combined_frames = pd.concat(lst_f)
    combined_frames.reset_index(inplace=True)
    # scale all values with the specified scaler
    combined_frames[norm_features + norm_mod_features] = scaler.fit_transform(combined_frames[nn_features + nnm_features])
    
    # split up the df for each mode again after rescaling
    f_car = combined_frames.iloc[:lst_idx[0], :].reset_index(drop=True)
    f_bike = combined_frames.iloc[lst_idx[0]:lst_idx[1], :].reset_index(drop=True)
    f_walk = combined_frames.iloc[lst_idx[1]:lst_idx[2], :].reset_index(drop=True)
    f_public = combined_frames.iloc[lst_idx[2]:lst_idx[3], :].reset_index(drop=True)
    lst_f = [f_car, f_bike, f_walk, f_public]
    
    for count, f in enumerate(lst_f):
        
        # transform combine all cost attributes into one column as a tuple first
        # then transform the tuples into a dict
        f["all_cost"] = f.apply(lambda x: (-x["norm_co2"], -x["norm_travel_time"], -x["norm_discomfort"], -x["norm_price"]), axis=1)
        f["all_cost"] = f["all_cost"].apply(lambda t: {mode[count]: {"norm_co2": t[0],
                                                                     "norm_travel_time": t[1],
                                                                     "norm_discomfort": t[2],
                                                                     "norm_price": t[3]
                                                                     }})
        f["true_cost"] = f.apply(lambda x: (-x["co2"], -x["travel_time"], -x["discomfort"], -x["price"]), axis=1)
        f["true_cost"] = f["true_cost"].apply(lambda t: {mode[count]: {"co2": t[0],
                                                                       "travel_time": t[1],
                                                                       "discomfort": t[2],
                                                                       "price": t[3]
                                                                       }})
        # add rescaled speed for each mode of transportation                                                             
        f["speed_ms"] = f["speed_ms"].apply(lambda spd: {mode[count]: spd})
        
        f.drop(beta_features + nn_features + nnm_features + norm_mod_features, axis=1, inplace=True)
    return lst_f
        
def combine_edges(frames):
    edges_drive, edges_bike, edges_walk, edges_public = frames[0], frames[1], frames[2], frames[3]
    # unique identifier for an edge --> osmid is not unique for each edge
    keys = ["u", "v", "key"]
    att = "all_cost"
    true_att = "true_cost"
    spd_att = "speed_ms"
    col_idx = len(frames[0].columns)

    # cars & bike --> not null values of the left join
    # only cars --> null values of the left join
    all_edges = pd.merge(edges_drive, edges_bike, how="left", on=keys, suffixes=("", "_right"))

    # get all edges which are available for cars and bikes 
    r = all_edges[pd.notnull(all_edges["osmid_right"])]

    # for all edges which are also bike routes the bike dictionary will be appended to the current dict
    [r.loc[x, att].update(r.loc[x, att + "_right"]) for x in r.index]
    [r.loc[x, true_att].update(r.loc[x, true_att + "_right"]) for x in r.index]
    [r.loc[x, spd_att].update(r.loc[x, spd_att + "_right"]) for x in r.index]                                                                   

    # cars & bike --> not null values of the left join
    # only bikes --> null values of the left join
    only_bike = pd.merge(edges_bike, edges_drive, how="left", on=keys, suffixes=("", "_right"))

    # get alle edges which are only available for bikes
    only_bike = only_bike[only_bike["osmid_right"].isnull()].copy()

    # combine the base edges with the bike only edges
    all_edges = pd.concat([all_edges, only_bike]).iloc[:, :col_idx]
    
    # available for walk --> not null values of the left join
    # not available for walk --> null values of the left join
    all_edges_plus_walk = pd.merge(all_edges, edges_walk, how="left", on=keys, suffixes=("", "_right"))

    # get all edges which are available for walk 
    w = all_edges_plus_walk[pd.notnull(all_edges_plus_walk["osmid_right"])]

    # for all edges which are also bike routes the bike dictionary will be appended to the current dict
    [w.loc[x, att].update(w.loc[x, att + "_right"]) for x in w.index]
    [w.loc[x, true_att].update(w.loc[x, true_att + "_right"]) for x in w.index]
    [w.loc[x, spd_att].update(w.loc[x, spd_att + "_right"]) for x in w.index]

    # only walks --> null values of the left join
    only_walk = pd.merge(edges_walk, all_edges, how="left", on=keys, suffixes=("", "_right"))

    # get alle edges which are only available for walks
    only_walk = only_walk[only_walk["osmid_right"].isnull()].copy()

    # combine the base edges with the walk only edges
    all_edges_plus_walk = pd.concat([all_edges_plus_walk, only_walk]).iloc[:, :col_idx]
    
    # available for public --> not null values of the left join
    # not available for public --> null values of the left join
    all_edges_plus_public = pd.merge(all_edges_plus_walk, edges_public, how="left", on=keys, suffixes=("", "_right"))

    # get all edges which are available for public 
    p = all_edges_plus_public[pd.notnull(all_edges_plus_public["osmid_right"])]

    # for all edges which are also public routes the public dictionary will be appended to the current dict
    [p.loc[x, att].update(p.loc[x, att + "_right"]) for x in p.index]
    [p.loc[x, true_att].update(p.loc[x, true_att + "_right"]) for x in p.index]
    [p.loc[x, spd_att].update(p.loc[x, spd_att + "_right"]) for x in p.index]
    # only public --> null values of the left join
    only_public = pd.merge(edges_public, all_edges_plus_walk, how="left", on=keys, suffixes=("", "_right"))

    # get alle edges which are only available for public
    only_public = only_public[only_public["osmid_right"].isnull()].copy()
#     print("before public", all_edges_plus_walk.shape)
#     print("Both", p.shape)
#     print("Only public", only_public.shape)

    # combine the base edges with the public only edges
    all_edges_plus_public = pd.concat([all_edges_plus_public, only_public]).iloc[:, :col_idx]
    return all_edges_plus_public.reset_index(drop=True)    

def full_transform_edges(frames, scaler):
    df_edges = combine_edges(select_add_transform_attributes(frames, scaler))
    df_edges["edge_id"] = df_edges["u"].astype(str) + "," + df_edges["v"].astype(str) + "," + df_edges["key"].astype(str)
    df_edges["edge_id"] = df_edges["edge_id"].apply(lambda s: tuple([int(k) for k in s.split(",")]))
    return df_edges

################ node transformations ################

# initiate all nodes with maximum pathcost, not explored and no previous node
def initiate_nodes(df_nodes):
    df_nodes["f_cost"] = sys.maxsize
    df_nodes["path_cost"] = 0.0
    df_nodes["heuristic"] = 0.0
    df_nodes["num_trans"] = 0.0
    df_nodes["norm_num_trans"] = 0.0
    df_nodes["max_trans"] = 0.0
    df_nodes["previous"] = None
    df_nodes["explored"] = False
    df_nodes["frontier"] = False
    df_nodes["trans_mode"] = None
    df_nodes["edge_idx"] = None
    for f in nnm_features:
        for m in mode:
            df_nodes["norm" + "_" + f + "_" + m] = 0.0
            df_nodes[f + "_" + m] = 0.0       
    for nn_f in nn_features:
        df_nodes["norm" + "_" + nn_f] = 0.0
        df_nodes[nn_f] = 0.0


# for each vertex "u" add every edge which starts from "u" to an adjacent list 
def add_adjacent(df_nodes, df_edges):
    adj = pd.merge(df_nodes, df_edges, left_on="osmid", right_on="u", suffixes=("_node", "_edge"))
    # each edge_id is added to the adjacent list of the node
    adj_edges = adj.groupby("osmid_node")["edge_id"].apply(lambda x: list(x))
    df_nodes["adjacent"] = adj_edges.copy()
    df_nodes["adjacent"] = df_nodes["adjacent"].apply(lambda x: tran_mult_edges(x))
        
def full_transform_nodes(frames, df_edges):
    for f in frames:
        f.drop(["highway", "ref"], axis=1, inplace=True, errors='ignore')
    df_c = pd.concat(frames)
    df_c.drop_duplicates(inplace=True)
    initiate_nodes(df_c)
    add_adjacent(df_c, df_edges)
    return df_c

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = MinMaxScaler()
all_edges = full_transform_edges(lst_edges, scaler)
all_edges.drop("index", axis=1, inplace=True)
all_edges.info()

In [ ]:
all_nodes = full_transform_nodes(lst_nodes, all_edges)
all_nodes.info()

## Check edge and node network

In [ ]:
all_edges.info()

In [ ]:
# all_nodes.info()

In [ ]:
# all_edges.head()

In [ ]:
# all_nodes.head()

In [ ]:
# take a look at the all_cost dictionary
all_edges["all_cost"].iloc[50]

In [ ]:
# take a look at the all_cost dictionary
all_edges["true_cost"].iloc[50]

# Heuristic

## Selecting target ids which are contained in all networks

In [ ]:
lst_sel_trg_id = []
# list of list of ids for each network
lst_ids_mode =  [list(nx.get_node_attributes(g, "osmid").values()) for g in lst_g]
# list of all node ids in the entire network
lst_id = list(all_nodes.osmid)

# length equals to number of observations
while len(lst_sel_trg_id) < num_obs:
    rnd_gen_base = 60275
    rnd_seeds = [i*rnd_gen_base for i in range(1,num_obs+1)]
    for seed in rnd_seeds:
        exist = True
        np.random.seed(seed)
        trg_id = lst_id.pop(np.random.randint(1, len(lst_id)))
        # check that the trg_id is in every network to minimize the chance that a node can not be reached
        for ids_mode in lst_ids_mode:
            if trg_id not in ids_mode:
                exist = False
                break
        if exist:
            lst_sel_trg_id.append(trg_id)
    rnd_gen_base += 1

# for g_node in lst_nodes

lst_sel_trg_id

## Heuristic distance calculation

In [ ]:
def dist_heuristic(src_id, trg_id):
    src_lat, src_lon = all_nodes.loc[src_id, ["lat", "lon"]]
    trg_lat, trg_lon = all_nodes.loc[trg_id, ["lat", "lon"]]
    dist_to_trg = ox.distance.great_circle_vec(src_lat, src_lon, trg_lat, trg_lon, earth_radius=6371009)
    return dist_to_trg

In [ ]:
# using the built in dijsktra algorithm to calculate the shortest distance to each node
# lst_sel_trg_id: list containing all trg ids
# G_all: Multigraph containing the whole network

# dict of dictionaries containing the distance of each src_node to each target node
lst_dist_to_trg = []
lst_id = list(nx.get_node_attributes(G_all, "osmid").values())

for count, trg_id in enumerate(lst_sel_trg_id):
    clear_output(wait=True)
    dist_to_trg = {}
    # calculate the shortest great-circle distance of each src_id to the trg_id
    # it is always underapproximative since it is the shortest distance possible to the trg_id
    for src_id in lst_id:
        shortest_dist = dist_heuristic(src_id, trg_id)
        dist_to_trg[src_id] = shortest_dist
    print(f"Remaining number of trg_ids: {len(lst_sel_trg_id) - (count+1)}")
    lst_dist_to_trg.append(dist_to_trg)

lst_dist_to_trg;

# Save all necessary files

In [ ]:
# save the lst_g as pickle file
g_file_name = os.path.join(prep_path, "lst_g4_100")
try: 
    scaler_file = open(g_file_name, 'wb') 
    pickle.dump(lst_g, scaler_file) 
    scaler_file.close() 
except: 
    print("Something went wrong")

In [ ]:
# save the scaler as pickle file
mm_file_name = os.path.join(prep_path, "min_max4_100")
try: 
    scaler_file = open(mm_file_name, 'wb') 
    pickle.dump(scaler, scaler_file) 
    scaler_file.close() 
except: 
    print("Something went wrong")

In [ ]:
# save edge and node network as pickle files
node_path = os.path.join(prep_path, "all_nodes.pkl")
edge_path = os.path.join(prep_path, "all_edges.pkl")

if not os.path.exists(prep_path):
    os.makedirs(prep_path)
    
all_nodes.to_pickle(node_path)
all_edges.to_pickle(edge_path)

In [ ]:
# save precalculated dict to file
trg_file_name = os.path.join(prep_path, "trg4_100")
try: 
    geeky_file = open(trg_file_name, 'wb') 
    pickle.dump(lst_sel_trg_id, geeky_file) 
    geeky_file.close() 
except: 
    print("Something went wrong")

In [ ]:
# save precalculated dict to file
dict_file_name = os.path.join(prep_path, "dist_to_trg4_100")
try: 
    geeky_file = open(dict_file_name, 'wb') 
    pickle.dump(lst_dist_to_trg, geeky_file) 
    geeky_file.close() 
except: 
    print("Something went wrong")

In [ ]:
# save G_all
# g_all_file_name = os.path.join(composed_graph_path, "G_all.graphml")
# ox.io.save_graphml(G_all, g_all_file_name)